In [1]:
import os
import numpy as np
import pandas as pd
from datetime import datetime
from src.defs import BROKER
from src.utils import logger, load_oanda_parquet
from src.notifications import log_order, log_trade
from backtrader_bokeh import bt

os.environ["BOKEH_ALLOW_WS_ORIGIN"] = '0lj9hh483va927cadklatiu5affjj5cn3b349he4cl71d36qc4p6'

logger.setLevel(10) # debug



***************: The token is incorrect, you will not be able to use advanced features, such as: 'logger', 'configer', 'keyboard control' and more... 

You can purchase Bakctrader_Bokeh on website: https://aui.photos/backtrader-bokeh/purchase/ 



Loading BokehJS ...

In [2]:
%load_ext autoreload
%autoreload 2

### Strategy

Ref: https://www.youtube.com/watch?v=fT4jb-I5zYc&t=8s

Long Signal:
1. NMACD cross over
2. RSI cross over
3. At the bar of RSI cross over, the close price is above SMA

In [3]:
INSTRUMENT = "EUR_USD"
DIGITS = BROKER.PRICE_DIGITS[INSTRUMENT]

class MyStrategy(bt.Strategy):
    params = (
        ('macd_fast_period', 13),
        ('macd_slow_period', 21),
        ('macd_signal_period', 9), # signal / trigger
        ('macd_normalize_period', 50),
        ('rsi_period', 21),
        ('rsi_ma_period', 55),
        ('ma_period', 13),
        ('atr_period', 115),
    )

    def __init__(self):

        # indicators
        self.sma = bt.indicators.SimpleMovingAverage(period=self.params.ma_period)

        self.macd = bt.indicators.MACD(
            period_me1=self.params.macd_fast_period,
            period_me2=self.params.macd_slow_period,
            period_signal=self.params.macd_signal_period
            )

        self.rsi = bt.indicators.RSI(period=self.params.rsi_period)
        self.rsi_ma = bt.indicators.SimpleMovingAverage(self.rsi, period=self.params.rsi_ma_period)
        self.rsi_ma.plotinfo.plotmaster = self.rsi
        
        self.atr = bt.indicators.AverageTrueRange(period=self.params.atr_period)

        # signals
        self.macd_crossover = bt.indicators.CrossOver(self.macd.macd, self.macd.signal, plot=False) # 0: no cross-over | 1: cross-up | -1 : cross-down
        self.rsi_crossover = bt.indicators.CrossOver(self.rsi, self.rsi_ma, plot=False) # 0: no cross-over | 1: cross-up | -1 : cross-down
        self.sig_macd_cross = 0 # 0: no signal | 1: cross above | -1: cross below
        self.signal = 0 # 0: no signal | 1: long | -1: short

        # prices
        self.stop_loss = None
        self.take_profit = None

        # others
        self.close_price = None
        self.order = None

    def update_signal(self):
        if self.macd_crossover[0] != 0:
            self.sig_macd_cross = self.macd_crossover[0]
        
        if self.sig_macd_cross==1:
            if (self.rsi_crossover[0]==1) & (self.data.close[0] > self.sma[0]):
                self.signal = 1
            else:
                self.signal = 0

        if self.sig_macd_cross==-1:
            if (self.rsi_crossover[0]==-1) & (self.data.close[0] < self.sma[0]):
                self.signal = -1
            else:
                self.signal = 0

    def next(self):
        #### trouble shooting ####
        #global g1
        #g1 = self.atr
        #self.log(self.atr[0])
        ####

        # if there is a pending order, do not send a 2nd one
        if self.order:
            return
    
        # update signal
        self.update_signal()

        # apply signal if not in position
        if not self.position:
            if self.signal == 1:
                self.log(f"BUY MKT ORDER TRIGGERED.")
                self.stop_loss = round(self.data.close[0] - self.atr[0], DIGITS)
                self.take_profit = round(self.data.close[0] + self.atr[0], DIGITS)
                self.order = self.buy_bracket(
                    size=None,
                    exectype=bt.Order.Market,
                    stopprice=self.stop_loss,
                    stopexec=bt.Order.Stop,
                    limitprice=self.take_profit,
                    limitexec=bt.Order.Limit,
                )
            if self.signal == -1:
                self.log(f"SELL MKT ORDER TRIGGERED.")
                self.stop_loss = round(self.data.close[0] + self.atr[0], DIGITS)
                self.take_profit = round(self.data.close[0] - self.atr[0], DIGITS)
                self.order = self.sell_bracket(
                    size=None,
                    exectype=bt.Order.Market,
                    stopprice=self.stop_loss,
                    stopexec=bt.Order.Stop,
                    limitprice=self.take_profit,
                    limitexec=bt.Order.Limit,
                )

    def notify_order(self, order: bt.order.Order) -> None:

        log_order(order=order, digits=DIGITS, log_func=self.log)
    
        # no pending order unless partially filled
        if order.status != order.Partial:
            self.order = None

    def notify_trade(self, trade: bt.trade.Trade) -> None:

        log_trade(trade=trade, digits=DIGITS, log_func=self.log)

    def start(self):
        self.log(f"STRATEGY START: value = {round(self.broker.getvalue(), DIGITS)}, cash = {round(self.broker.getcash(), DIGITS)}.", with_dt=False)
    
    def stop(self):
        self.log(f"STRATEGY COMPLETE: value = {round(self.broker.getvalue(), DIGITS)}, cash = {round(self.broker.getcash(), DIGITS)}.")

    def log(self, txt, with_dt=True):
        if with_dt:
            dt = self.data.datetime.datetime(0)
            txt = f"[{dt.strftime('%Y-%m-%d %H:%M:%S')}] {txt}"
        print(txt)


In [5]:
DATA_FILE = 'oanda_EUR_USD_M1_2022-12-19_2022-12-31.parquet.gz'
DATA_NAME = 'EUR_USD_H1'
TIMEFRAME = bt.TimeFrame.Minutes
COMPRESSION = 60

# initialize
cerebro = bt.Cerebro()
cerebro.broker.setcash(1000)
cerebro.broker.setcommission(commission=0.0)

# add data
df = load_oanda_parquet(DATA_FILE)

data = bt.feeds.PandasData(
    dataname=df,
    name=DATA_NAME,
)

cerebro.replaydata(data, timeframe=TIMEFRAME, compression=COMPRESSION)

# add strategy
cerebro.addstrategy(MyStrategy)

# add sizer
cerebro.addsizer(bt.sizers.FixedSize, stake=100)

# add analyzers
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='Trades')
cerebro.addanalyzer(bt.analyzers.Returns, _name='Returns')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DrawDown')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio')

results = cerebro.run()
result = results[0]


[utils.py:38 - load_oanda_parquet()] Loaded data has 12741 rows, from 2022-12-19 00:00:00+00:00 to 2022-12-30 21:58:00+00:00.


STRATEGY START: value = 1000, cash = 1000.
[2022-12-27 12:54:00] SELL MKT ORDER TRIGGERED.
[2022-12-27 12:55:00] SELL MKT ORDER ACCEPTED: price = 1.06382, size = 100.
[2022-12-27 12:55:00] BUY STOP ORDER ACCEPTED: price = 1.06554, size = 100.
[2022-12-27 12:55:00] BUY LMT ORDER ACCEPTED: price = 1.0621, size = 100.
[2022-12-27 12:55:00] SELL MKT ORDER EXECUTED: order_price = 1.06382, execute_price = 1.06381, slippage = 0.0 %%, size = 100, value = 106.3815, commission = 0.0.
[2022-12-27 13:11:00] BUY LMT ORDER EXECUTED: order_price = 1.0621, execute_price = 1.0621, slippage = 0.0 %%, size = 100, value = 106.21, commission = 0.0.
[2022-12-27 13:11:00] BUY STOP ORDER CANCELLED.
[2022-12-27 13:11:00] TRADE 1 CLOSED: gross_pnl = 0.1715, net_pnl = 0.1715.
[2022-12-27 17:33:00] SELL MKT ORDER TRIGGERED.
[2022-12-27 17:34:00] SELL MKT ORDER ACCEPTED: price = 1.06375, size = 100.
[2022-12-27 17:34:00] BUY STOP ORDER ACCEPTED: price = 1.06549, size = 100.
[2022-12-27 17:34:00] BUY LMT ORDER ACCE

In [5]:
bullish_color = '#31a354' # "green" 
bearish_color = '#e6550d' # "red"
bokeh_plotter = bt.Bokeh(barup=bullish_color, barup_outline=bullish_color, barup_wick=bullish_color, 
                         bardown=bearish_color, bardown_outline=bearish_color, bardown_wick=bearish_color)
cerebro.plot(plotter=bokeh_plotter, iplot=False)

[server.py:403 - __init__()] Starting Bokeh server version 2.4.3 (running on Tornado 6.2)
[tornado.py:360 - __init__()] User authentication hooks NOT provided (default user enabled)


[{0: <backtrader_bokeh.figure.FigurePage at 0x1ce354158b0>}]

[web.py:2271 - log_request()] 200 GET /autoload.js?bokeh-autoload-element=1595&bokeh-absolute-url=http://localhost:62835&resources=none (::1) 445.16ms
[web.py:2271 - log_request()] 101 GET /ws?id=31fc6055-7028-4a9f-aabf-41836537a557&origin=bce4bbf1-101d-44db-883b-e3db4f31e0a2&swVersion=4&extensionId=&platform=electron&vscode-resource-base-authority=vscode-resource.vscode-cdn.net&parentOrigin=vscode-file%3A%2F%2Fvscode-app&purpose=notebookRenderer (::1) 16.53ms
[ws.py:132 - open()] WebSocket connection opened
[ws.py:213 - _async_open()] ServerConnection created


In [ ]:
print('Total Trades:', result.analyzers.Trades.get_analysis()['total'])
print('Total Returns:', result.analyzers.Returns.get_analysis()['rtot'])
print('Max Drawdown:', result.analyzers.DrawDown.get_analysis()['max']['drawdown'])
print('Sharpe Ratio:', result.analyzers.SharpeRatio.get_analysis()['sharperatio'])

In [ ]:
DATA_FILE = 'oanda_EUR_USD_H1_2022-12-19_2022-12-31.parquet.gz'

# initialize
cerebro = bt.Cerebro()
cerebro.broker.setcash(1000)
cerebro.broker.setcommission(commission=0.0)

# add data
df = load_oanda_parquet(DATA_FILE)

data = bt.feeds.PandasData(
    dataname=df,
    name='EUR_USD_H1',
)

cerebro.adddata(data)

# add strategy
cerebro.addstrategy(MyStrategy)

# add sizer
cerebro.addsizer(bt.sizers.FixedSize, stake=100)

# add analyzers
cerebro.addanalyzer(bt.analyzers.TradeAnalyzer, _name='Trades')
cerebro.addanalyzer(bt.analyzers.Returns, _name='Returns')
cerebro.addanalyzer(bt.analyzers.DrawDown, _name='DrawDown')
cerebro.addanalyzer(bt.analyzers.SharpeRatio, _name='SharpeRatio')

results = cerebro.run()
result = results[0]
